## Evaluate Model on Holdout

In [1]:
import os
#import s3fs
import time
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
np.set_printoptions(precision=4)

import catboost
#print(catboost.__version__)

from catboost import Pool
from catboost import CatBoostClassifier

from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt   

In [2]:
# helper function for model evaluation
def eval(X_holdout, y_holdout, model):

    dholdout_pred = model.predict(X_holdout)
    dholdout_predprob = model.predict_proba(X_holdout)[:,1]
    measure = metrics.precision_recall_fscore_support(y_holdout, 
                                                      dholdout_pred, 
                                                      beta=1.0, 
                                                      labels=None, 
                                                      pos_label=1, 
                                                      average='binary')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_holdout, dholdout_pred))
    print ("AUC Score (Holdout): %f" % metrics.roc_auc_score(y_holdout, dholdout_pred))
    print("Confusion Matrix (Actual X Pred) :\n", metrics.confusion_matrix(y_holdout, dholdout_pred))
    print ("Precision : %f" % measure[0])
    print("Recall : %f"  % measure[1])

In [4]:
dholdout = pd.read_csv('C:/Users/n1507890/Documents/Python2-master/WC_DeEscl_Data/2-Data_Construction/WC_DeEscl_Data_Holdout_wClstr_v2_dholdout_O.csv')
dholdout.head(10)

Year  Claimant_Age_Qty  Tenure  Avg_Wkly_Wge_Amt  InjurytoReport  \
0  2013                44     257            390.00               1   
1  2013                31     626            179.12               1   
2  2013                22     279             47.15               1   
3  2013                42    5597            931.45             323   
4  2013                58     674            380.00               9   
5  2013                59    2169            518.40               1   
6  2013                48     381            520.00               2   
7  2013                37     458            380.00               1   
8  2013                53     419            200.00               3   
9  2013                43     367              5.50               3   

   ReporttoEntry  Dist_to_work  prklot_idx  prklot_strain_sprain  \
0              2      0.000000           0                     1   
1              1   2101.012441           0                     0   
2              1    988.972873           0                     0   
3              1      0.000000           0                     1   
4              1      0.000000           0                     1   
5              1      0.000000           0                     1   
6              1      0.000000           0                     0   
7              1      0.000000           0                     0   
8              1      0.000000           0                     1   
9              1      0.000000           0                     1   

   prklot_head_category  ...  Occur_pam_9c_4  Occur_pam_9c_5  Occur_pam_9c_6  \
0                     0  ...               0               0               0   
1                     0  ...               0               1               0   
2                     0  ...               0               1               0   
3                     0  ...               0               1               0   
4                     0  ...               0               0               1   
5                     0  ...               0               0               0   
6                     0  ...               0               1               0   
7                     0  ...               0               1               0   
8                     0  ...               0               0               1   
9                     0  ...               0               0               1   

   Occur_pam_9c_7  Occur_pam_9c_8  Occur_pam_9c_9  \
0               0               0               1   
1               0               0               0   
2               0               0               0   
3               0               0               0   
4               0               0               0   
5               0               0               1   
6               0               0               0   
7               0               0               0   
8               0               0               0   
9               0               0               0   

   Occur_pam_9c_Crash of airplane  Occur_pam_9c_Vehicle upset (overturned)  \
0                               0                                        0   
1                               0                                        0   
2                               0                                        0   
3                               0                                        0   
4                               0                                        0   
5                               0                                        0   
6                               0                                        0   
7                               0                                        0   
8                               0                                        0   
9                               0                                        0   

   Prim_NOI_Dsc_Enucleation (surgical removal of a tumor, eye, etc.)  \
0                                                  0                   
1   

In [5]:
X_holdout_onehot = dholdout.iloc[:, :-1]
y_holdout = dholdout.iloc[:, -1]

In [6]:
print(X_holdout_onehot.shape)
print(y_holdout.shape)

(136131, 617)
(136131,)


In [7]:
import joblib

model = joblib.load('C:/Users/n1507890/Documents/Python2-master/WC_DeEscl_Data/5-Scoring/catboost_model_rs.pkl') 

C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 0.21.2 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
dholdout_predprob = model.predict_proba(X_holdout_onehot)[:,1]
dholdout_pred = np.where(dholdout_predprob >= 0.75, 1, 0)

In [9]:
mtx = metrics.confusion_matrix(y_holdout, dholdout_pred)
pr = mtx[1,1]/(mtx[0,1] + mtx[1,1])
rc = mtx[1,1]/(mtx[1,0] + mtx[1,1])

print("Recall : %f"  % rc)
print ("Precision : %f" % pr)

Recall : 0.659378
Precision : 0.891805


In [10]:
mtx

array([[28896,  7943],
       [33821, 65471]], dtype=int64)